In [ ]:
from typing import List, Optional, Generator
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect
import heapq

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str, dtypes: dict[str, str]=dict()) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    temp = temp.astype(dtypes)
    return temp

conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [ ]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)

        while visited and not visited[-1]:
            visited.pop()

        return visited

    def __str__(self):
        return str(self.val)

    @staticmethod
    def from_array(nums: list[int|None]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)

                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

#### Helper for Singly Linked lists

In [ ]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    @staticmethod
    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr

        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

#### Matrix factorization GYM Practice: https://codeforces.com/gym/102644

In [ ]:
def count_paths_brute(N: int, E: int, K: int, edges: list[tuple[int, int]]) -> int:
    @functools.cache
    def backtrack(curr: int, k: int) -> int:
        if k == 0:
            return 1
        else:
            result = 0
            for next_node in adjl[curr]:
                result += backtrack(next_node, k - 1)
            return result % MOD

    # Convert edges into adjl
    adjl: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for n1, n2 in edges:
        adjl[n2].append(n1)

    # Do a 'cached' DFS
    MOD = 10 ** 9 + 7
    return sum(backtrack(i, K) for i in range(1, N + 1)) % MOD

# Testing the solution
assert count_paths_brute(3, 4, 2, [(1,2), (2,3), (3,1), (2,1)]) == 5
assert count_paths_brute(5, 10, 11, [(2,3), (4,2), (2,1), (2,4), (1,5), (5,2), (3,2), (3,1), (3,4), (1,2)]) == 21305

Atcoder Beginner contest: 365

In [ ]:
def leap_year(year: int) -> int:
    if year % 4 != 0:
        return 365
    elif year % 100 != 0:
        return 366
    elif year % 400 != 0:
        return 365
    else:
        return 366

# Testing the solution
assert leap_year(2023) == 365
assert leap_year(1992) == 366
assert leap_year(1800) == 365
assert leap_year(1600) == 366

# %$
def second_largest(N: int, nums: list[int]) -> int:
    max_ = max(nums)
    idx = 0 if nums[0] != max_ else 1
    for i in range(N):
        if nums[idx] < nums[i] < max_:
            idx = i
    return idx + 1

# Testing the solution
assert second_largest(5, [3,8,2,5,1]) == 4
assert second_largest(4, [8,2,5,1]) == 3
assert second_largest(2, [2,1]) == 2
assert second_largest(8, [1,2,3,4,5,10,9,11]) == 6

In [ ]:
def transportation_expenses(N: int, M: int, costs: list[int]) -> int:
    total_cost = sum(costs)
    if total_cost <= M:
        return -1
    else:
        low, high = 0, M
        while low <= high:
            mid = (low + high) // 2
            total_cost = sum(map(lambda x: min(x, mid), costs))
            if total_cost <= M:
                low = mid + 1
            else:
                high = mid - 1

        return high

# Testing the solution
assert transportation_expenses(4, 8, [1,3,2,4]) == 2
assert transportation_expenses(3, 20, [5,3,2]) == -1
assert transportation_expenses(10, 23, [2,5,6,5,2,1,7,9,7,2]) == 2

In [ ]:
def RPS(N: int, moves: str) -> int:

    win_mapping: dict[str, str] = {"R": "P", "S": "R", "P": "S"}
    move_mapping: dict[int, str] = dict(enumerate("RPS"))
    move_mapping_rev: dict[str, int] = {j: i for i, j in move_mapping.items()}

    dp: list[int] = [0 for j in range(3)]
    for i in range(N - 1, -1, -1):
        curr, win_move = moves[i], win_mapping[moves[i]]
        next_dp: list[int] = [0 for i in range(3)]
        for j in range(3):
            prev = move_mapping[j]
            if win_move != prev:
                next_dp[j] = max(next_dp[j], 1 + dp[move_mapping_rev[win_move]])
            if moves[i] != prev:
                next_dp[j] = max(next_dp[j], dp[move_mapping_rev[curr]])
        dp = next_dp

    return max(dp)

# Testing the solution
assert RPS(24, "SPRPSRRRRRPPRPRPSSRSPRSS") == 18
assert RPS(10, "SSSSSSSSSS") == 5
assert RPS(5, "SSPRS") == 4
assert RPS(6, "PRSSRS") == 5

#### Leetcode Biweekly

In [ ]:
def minFlips(grid: list[list[int]]) -> int:
    M, N = len(grid), len(grid[0])

    # Count flips required horizontally
    h_flips = 0
    for i in range(M):
        j, k, flips = 0, N - 1, 0
        while j < k:
            if grid[i][j] != grid[i][k]:
                flips += 1
            j, k = j + 1, k - 1
        h_flips += flips

    # Count flips required vertically
    v_flips = 0
    for i in range(N):
        j, k, flips = 0, M - 1, 0
        while j < k:
            if grid[j][i] != grid[k][i]:
                flips += 1
            j, k = j + 1, k - 1
        v_flips += flips

    return min(h_flips, v_flips)

# Testing the solution
assert minFlips([[1,0,0],[0,0,0],[0,0,1]]) == 2
assert minFlips([[0,1],[0,1],[0,0]]) == 1
assert minFlips([[1],[0]]) == 0
assert minFlips([[1]]) == 0